# Домашнее задание 3. Парсинг, Git и тестирование на Python

**Цели задания:**

* Освоить базовые подходы к web-scraping с библиотеками `requests` и `BeautisulSoup`: навигация по страницам, извлечение HTML-элементов, парсинг.
* Научиться автоматизировать задачи с использованием библиотеки `schedule`.
* Попрактиковаться в использовании Git и оформлении проектов на GitHub.
* Написать и запустить простые юнит-тесты с использованием `pytest`.


В этом домашнем задании вы разработаете систему для автоматического сбора данных о книгах с сайта [Books to Scrape](http://books.toscrape.com). Нужно реализовать функции для парсинга всех страниц сайта, извлечения информации о книгах, автоматического ежедневного запуска задачи и сохранения результата.

Важной частью задания станет оформление проекта: вы создадите репозиторий на GitHub, оформите `README.md`, добавите артефакты (код, данные, отчеты) и напишете базовые тесты на `pytest`.



In [1]:
#! pip install -q schedule pytest # установка библиотек, если ещё не

In [2]:
# Библиотеки, которые могут вам понадобиться
# При необходимости расширяйте список

# Стандартная библиотеки
import json
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
from typing import Dict, List, Optional, Union
from urllib.parse import urljoin

# Сторонние библиотеки
import requests
import schedule
from bs4 import BeautifulSoup, Tag
from requests.sessions import Session
from typing_extensions import TypeAlias


## Задание 1. Сбор данных об одной книге (20 баллов)

В этом задании мы начнем подготовку скрипта для парсинга информации о книгах со страниц каталога сайта [Books to Scrape](https://books.toscrape.com/).

Для начала реализуйте функцию `get_book_data`, которая будет получать данные о книге с одной страницы (например, с [этой](http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html)). Соберите всю информацию, включая название, цену, рейтинг, количество в наличии, описание и дополнительные характеристики из таблицы Product Information. Результат достаточно вернуть в виде словаря.

**Не забывайте про соблюдение PEP-8** — помимо качественно написанного кода важно также документировать функции по стандарту:
* кратко описать, что она делает и для чего нужна;
* какие входные аргументы принимает, какого они типа и что означают по смыслу;
* аналогично описать возвращаемые значения.


*P. S. Состав, количество аргументов функции и тип возвращаемого значения можете менять как вам удобно. То, что написано ниже в шаблоне — лишь пример.*

In [3]:
# НАЧАЛО ВАШЕГО РЕШЕНИЯ

# Псевдонимы типов
BookData: TypeAlias = Dict[str, Union[str, int, None]]
BookList: TypeAlias = List[BookData]

# Константы
RATING_MAP = {'One': 1, 'Two': 2, 'Three': 3, 'Four': 4, 'Five': 5}
DEFAULT_TIMEOUT = 10
MAX_WORKERS = 10


# pylint: disable=too-many-locals
def get_book_data(session: Session, book_url: str) -> BookData:
    """
    Извлекает данные о книге с указанной страницы

    Args:
        session (Session): Сессия requests для выполнения запроса
        book_url (str): URL страницы книги

    Returns:
        BookData: Словарь с данными о книге.
                  Возвращает пустой словарь в случае ошибки сети.
    """
    try:
        response = session.get(book_url, timeout=DEFAULT_TIMEOUT)
        response.raise_for_status()
        response.encoding = 'utf-8'
        soup = BeautifulSoup(response.text, 'html.parser')

        # Извлечение основных данных
        title_element = soup.find('h1')
        title = title_element.text.strip() if title_element else None

        price_element = soup.find('p', class_='price_color')
        price = price_element.text.strip() if price_element else None

        # Извлечение рейтинга
        rating = 0
        rating_element = soup.find('p', class_='star-rating')
        if isinstance(rating_element, Tag) and 'class' in rating_element.attrs:
            rating_classes = rating_element['class']
            if isinstance(rating_classes, list) and len(rating_classes) > 1:
                rating = RATING_MAP.get(rating_classes[1], 0)

        # Извлечение количества в наличии
        stock_element = soup.find('p', class_='instock availability')
        stock = stock_element.text.strip() if stock_element else None

        # Извлечение описания
        description = None
        description_header = soup.find('div', id='product_description')
        if description_header:
            description_element = description_header.find_next_sibling('p')
            description = description_element.text.strip() \
                if description_element else None

        # Извлечение характеристик из таблицы
        info = {}
        info_table = soup.find('table', class_='table table-striped')
        if isinstance(info_table, Tag):
            rows = info_table.find_all('tr')
            for row in rows:
                th_element = row.find('th')
                td_element = row.find('td')
                if th_element and td_element:
                    info_key = th_element.text.strip()
                    info_value = td_element.text.strip()
                    info[info_key] = info_value

        return {
            'title': title,
            'price': price,
            'rating': rating,
            'stock': stock,
            'description': description,
            'url': book_url,
            **info
        }

    except requests.RequestException as exc:
        print(f"Ошибка при загрузке {book_url}: {exc}")
        return {}

    # КОНЕЦ ВАШЕГО РЕШЕНИЯ

In [4]:
# Используйте для самопроверки
book_url = 'http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html'
# get_book_data(session, book_url)

# Создаем сессию и используем ее
with requests.Session() as session:
    book_data = get_book_data(session, book_url)
    print(json.dumps(book_data, indent=2, ensure_ascii=False))

{
  "title": "A Light in the Attic",
  "price": "£51.77",
  "rating": 3,
  "stock": "In stock (22 available)",
  "description": "It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love th It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love that Silverstein. Need proof of his genius? RockabyeRockabye baby, in the treetopDon't you know a treetopIs no safe place 

## Задание 2. Сбор данных обо всех книгах (20 баллов)

Создайте функцию `scrape_books`, которая будет проходиться по всем страницам из каталога (вида `http://books.toscrape.com/catalogue/page-{N}.html`) и осуществлять парсинг всех страниц в цикле, используя ранее написанную `get_book_data`.

Добавьте аргумент-флаг, который будет отвечать за сохранение результата в файл: если он будет равен `True`, то информация сохранится в ту же папку в файл `books_data.txt`; иначе шаг сохранения будет пропущен.

**Также не забывайте про соблюдение PEP-8**

In [5]:
# НАЧАЛО ВАШЕГО РЕШЕНИЯ
# Константы
BASE_URL = 'http://books.toscrape.com'
CATALOGUE_URL_TEMPLATE = 'http://books.toscrape.com/catalogue/page-{}.html'
DEFAULT_TIMEOUT = 10
MAX_WORKERS = 10

def scrape_books(is_save: bool = False,
                max_pages: Optional[int] = None) -> BookList:
    """
    Парсит все страницы каталога и собирает данные о книгах с использованием
    многопоточности

    Args:
        is_save (bool): Если True, сохраняет данные в файл books_data.txt
                       в папке artifacts/.
        max_pages (Optional[int]): Максимальное количество страниц для
                                  парсинга. Если None, парсит все страницы

    Returns:
        BookList: Список словарей с данными о книгах
    """
    print("Начало парсинга книг")

    with requests.Session() as session:
        session.headers.update({
            'User-Agent': 'MyBookScraper (student project)'
        })

        # Проверка доступности сайта
        try:
            response = session.get(BASE_URL, timeout=DEFAULT_TIMEOUT)
            response.raise_for_status()
            print(f"Сайт {BASE_URL} доступен")
        except requests.RequestException as exc:
            print(f"Сайт {BASE_URL} недоступен: {exc}")
            return []

        # Сбор всех URL книг
        all_book_urls = []
        page = 1

        while True:
            if max_pages and page > max_pages:
                break

            page_url = CATALOGUE_URL_TEMPLATE.format(page)

            try:
                response = session.get(page_url, timeout=DEFAULT_TIMEOUT)
                if response.status_code != 200:
                    print(f"Страница {page} недоступна. Завершение сбора URL.")
                    break

                response.encoding = 'utf-8'
                soup = BeautifulSoup(response.text, 'html.parser')
                book_links = soup.select('article.product_pod h3 a')

                if not book_links:
                    print(f"Страница {page} пуста. Завершение сбора URL.")
                    break

                for link in book_links:
                    href_value = link.get('href')

                    if isinstance(href_value, str):
                        book_url = urljoin(page_url, href_value)
                        all_book_urls.append(book_url)

                print(f"Найдено {len(book_links)} книг на странице {page}")
                page += 1

            except requests.RequestException as exc:
                print(f"Ошибка при загрузке страницы {page_url}: {exc}")
                break

        print(f"Всего найдено {len(all_book_urls)} книг для парсинга")

        if not all_book_urls:
            print("Не найдено ни одной книги для парсинга")
            return []

        parsed_books = []
        failed_count = 0
        total_books = len(all_book_urls)
        print(f"Запуск {MAX_WORKERS} потоков для обработки {total_books} книг...")

        with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
            futures = {
                executor.submit(get_book_data, session, url): url
                for url in all_book_urls
            }

            for i, future in enumerate(as_completed(futures), 1):
                url = futures[future]
                try:
                    book_data = future.result()
                    if book_data:
                        parsed_books.append(book_data)
                    else:
                        failed_count += 1
                        print(f"\n[Поток] Не удалось получить данные для: {url}")
                except Exception as exc:
                    failed_count += 1
                    print(f"\n[Поток] Ошибка при обработке {url}: {exc}")

                print(f"Обработано: {i}/{total_books} (Ошибок: {failed_count})",
                      end="\r")

    print()
    print("=" * 30)
    print("Парсинг завершен.")
    print(f"Успешно обработано {len(parsed_books)} книг, "
          f"неудачных попыток: {failed_count}")

    if is_save and parsed_books:
        save_books_data(parsed_books)

    return parsed_books

def save_books_data(book_list: BookList,
                   filename: str = 'books_data.txt') -> None:
    """
    Сохраняет данные о книгах в файл

    Args:
        book_list (BookList): Список данных о книгах
        filename (str): Имя файла для сохранения
    """
    try:
        filepath = filename

        with open(filepath, 'w', encoding='utf-8') as file:
            json.dump(book_list, file, ensure_ascii=False, indent=4)

        print(f"Данные о {len(book_list)} книгах сохранены в {filepath}")

    except (OSError, IOError) as exc:
        print(f"Ошибка при сохранении данных: {exc}")
    # КОНЕЦ ВАШЕГО РЕШЕНИЯ

In [6]:
# Проверка работоспособности функции
res = scrape_books(is_save=True)
print(type(res), len(res))

Начало парсинга книг
Сайт http://books.toscrape.com доступен
Найдено 20 книг на странице 1
Найдено 20 книг на странице 2
Найдено 20 книг на странице 3
Найдено 20 книг на странице 4
Найдено 20 книг на странице 5
Найдено 20 книг на странице 6
Найдено 20 книг на странице 7
Найдено 20 книг на странице 8
Найдено 20 книг на странице 9
Найдено 20 книг на странице 10
Найдено 20 книг на странице 11
Найдено 20 книг на странице 12
Найдено 20 книг на странице 13
Найдено 20 книг на странице 14
Найдено 20 книг на странице 15
Найдено 20 книг на странице 16
Найдено 20 книг на странице 17
Найдено 20 книг на странице 18
Найдено 20 книг на странице 19
Найдено 20 книг на странице 20
Найдено 20 книг на странице 21
Найдено 20 книг на странице 22
Найдено 20 книг на странице 23
Найдено 20 книг на странице 24
Найдено 20 книг на странице 25
Найдено 20 книг на странице 26
Найдено 20 книг на странице 27
Найдено 20 книг на странице 28
Найдено 20 книг на странице 29
Найдено 20 книг на странице 30
Найдено 20 книг на

## Задание 3. Настройка регулярной выгрузки (10 баллов)

Настройте автоматический запуск функции сбора данных каждый день в 19:00.
Для автоматизации используйте библиотеку `schedule`. Функция должна запускаться в указанное время и сохранять обновленные данные в текстовый файл.



Бесконечный цикл должен обеспечивать постоянное ожидание времени для запуска задачи и выполнять ее по расписанию. Однако чтобы не перегружать систему, стоит подумать о том, чтобы выполнять проверку нужного времени не постоянно, а раз в какой-то промежуток. В этом вам может помочь `time.sleep(...)`.

Проверьте работоспособность кода локально на любом времени чч:мм.



In [ ]:
# НАЧАЛО ВАШЕГО РЕШЕНИЯ
def run_scraper() -> None:
    """
    Функция для запуска парсера и сохранения данных в файл.
    """
    print(f"Запуск парсинга в {time.strftime('%H:%M:%S')}")
    try:
        scraped_books = scrape_books(is_save=True)
        if scraped_books:
            print(f"Парсинг завершен успешно. Обработано {len(scraped_books)} книг.")
        else:
            print("Парсинг не выполнен: нет данных")
    except Exception as exc:
        print(f"Ошибка при выполнении парсинга: {exc}")


def main_loop(target_time: str = "12:00", test_delay: int = 60) -> None:
    """
    Основной цикл для проверки и выполнения задач по расписанию.
    """
    schedule.every().day.at(target_time).do(run_scraper)
    print(f"Планировщик настроен на запуск каждый день в {target_time}")

    if test_delay:
        test_time_dt = time.localtime(time.time() + test_delay)
        test_time_str = time.strftime("%H:%M:%S", test_time_dt)
        schedule.every().day.at(test_time_str).do(run_scraper)
        print(f"Добавлен тестовый запуск в {test_time_str}")

    try:
        print("Запуск основного цикла планировщика")
        while True:
            schedule.run_pending()
            time.sleep(60)
    except KeyboardInterrupt:
        print("\nПолучен сигнал прерывания. Остановка планировщика.")
    except Exception as exc:
        print(f"Ошибка в основном цикле: {exc}")

if __name__ == "__main__":
    main_loop()
# КОНЕЦ ВАШЕГО РЕШЕНИЯ

Планировщик настроен на запуск каждый день в 12:00
Добавлен тестовый запуск в 22:14:30
Запуск основного цикла планировщика
Запуск парсинга в 22:14:30
Начало парсинга книг
Сайт http://books.toscrape.com доступен
Найдено 20 книг на странице 1
Найдено 20 книг на странице 2
Найдено 20 книг на странице 3
Найдено 20 книг на странице 4
Найдено 20 книг на странице 5
Найдено 20 книг на странице 6
Найдено 20 книг на странице 7
Найдено 20 книг на странице 8
Найдено 20 книг на странице 9
Найдено 20 книг на странице 10
Найдено 20 книг на странице 11
Найдено 20 книг на странице 12
Найдено 20 книг на странице 13
Найдено 20 книг на странице 14
Найдено 20 книг на странице 15
Найдено 20 книг на странице 16
Найдено 20 книг на странице 17
Найдено 20 книг на странице 18
Найдено 20 книг на странице 19
Найдено 20 книг на странице 20
Найдено 20 книг на странице 21
Найдено 20 книг на странице 22
Найдено 20 книг на странице 23
Найдено 20 книг на странице 24
Найдено 20 книг на странице 25
Найдено 20 книг на стра

## Задание 4. Написание автотестов (15 баллов)

Создайте минимум три автотеста для ключевых функций парсинга — например, `get_book_data` и `scrape_books`. Идеи проверок (можете использовать свои):

* данные о книге возвращаются в виде словаря с нужными ключами;
* список ссылок или количество собранных книг соответствует ожиданиям;
* значения отдельных полей (например, `title`) корректны.

Оформите тесты в отдельном скрипте `tests/test_scraper.py`, используйте библиотеку `pytest`. Убедитесь, что тесты проходят успешно при запуске из терминала командой `pytest`.

Также выведите результат их выполнения в ячейке ниже.


**Не забывайте про соблюдение PEP-8**


In [1]:
# Ячейка для демонстрации работоспособности
# Сам код напишите в отдельном скрипте
!pytest ../tests/test_scraper.py -v


============================= test session starts =============================
platform win32 -- Python 3.13.3, pytest-8.4.2, pluggy-1.6.0 -- c:\Python\Lessons 3\.venv\Scripts\python.exe
cachedir: .pytest_cache
rootdir: C:\Python\Lessons 3
plugins: anyio-4.11.0, cov-4.1.0
collecting ... collected 3 items

..\tests\test_scraper.py::test_get_book_data_parsing PASSED              [ 33%]
..\tests\test_scraper.py::test_scrape_books_page_limit PASSED            [ 66%]
..\tests\test_scraper.py::test_get_book_data_network_error PASSED        [100%]

============================== 3 passed in 3.75s ==============================


## Задание 5. Оформление проекта на GitHub и работа с Git (35 баллов)

В этом задании нужно воспользоваться системой контроля версий Git и платформой GitHub для хранения и управления своим проектом. **Ссылку на свой репозиторий пришлите в форме для сдачи ответа.**

### Пошаговая инструкция и задания

**1. Установите Git на свой компьютер.**

* Для Windows: [скачайте установщик](https://git-scm.com/downloads) и выполните установку.
* Для macOS:

  ```
  brew install git
  ```
* Для Linux:

  ```
  sudo apt update
  sudo apt install git
  ```

**2. Настройте имя пользователя и email.**

Это нужно для подписи ваших коммитов, сделайте в терминале через `git config ...`.

**3. Создайте аккаунт на GitHub**, если у вас его еще нет:
[https://github.com](https://github.com)

**4. Создайте новый репозиторий на GitHub:**

* Найдите кнопку **New repository**.
* Укажите название, краткое описание, выберите тип **Public** (чтобы мы могли проверить ДЗ).
* Не ставьте галочку Initialize this repository with a README.

**5. Создайте локальную папку с проектом.** Можно в терминале, можно через UI, это не имеет значения.

**6. Инициализируйте Git в этой папке.** Здесь уже придется воспользоваться некоторой командой в терминале.

**7. Привяжите локальный репозиторий к удаленному на GitHub.**

**8. Создайте ветку разработки.** По умолчанию вы будете находиться в ветке `main`, создайте и переключитесь на ветку `hw-books-parser`.

**9. Добавьте в проект следующие файлы и папки:**

* `scraper.py` — ваш основной скрипт для сбора данных.
* `README.md` — файл с кратким описанием проекта:

  * цель;
  * инструкции по запуску;
  * список используемых библиотек.
* `requirements.txt` — файл со списком зависимостей, необходимых для проекта (не присылайте все из глобального окружения, создайте изолированную виртуальную среду, добавьте в нее все нужное для проекта и получите список библиотек через `pip freeze`).
* `artifacts/` — папка с результатами парсинга (`books_data.txt` — полностью или его часть, если весь не поместится на GitHub).
* `notebooks/` — папка с заполненным ноутбуком `HW_03_python_ds_2025.ipynb` и запущенными ячейками с выводами на экран.
* `tests/` — папка с тестами на `pytest`, оформите их в формате скрипта(-ов) с расширением `.py`.
* `.gitignore` — стандартный файл, который позволит исключить временные файлы при добавлении в отслеживаемые (например, `__pycache__/`, `.DS_Store`, `*.pyc`, `venv/` и др.).


**10. Сделайте коммит.**

**11. Отправьте свою ветку на GitHub.**

**12. Создайте Pull Request:**

* Перейдите в репозиторий на GitHub.
* Нажмите кнопку **Compare & pull request**.
* Укажите, что было добавлено, и нажмите **Create pull request**.

**13. Выполните слияние Pull Request:**

* Убедитесь, что нет конфликтов.
* Нажмите **Merge pull request**, затем **Confirm merge**.

**14. Скачайте изменения из основной ветки локально.**



### Требования к итоговому репозиторию

* Файл `scraper.py` с рабочим кодом парсера.
* `README.md` с описанием проекта и инструкцией по запуску.
* Папка `artifacts/` с результатом сбора данных (`.txt` файл).
* Папка `tests/` с тестами на `pytest`.
* Папка `notebooks/` с заполненным ноутбуком `HW_03_python_ds_2025.ipynb`.
* Pull Request с комментарием из ветки `hw-books-parser` в ветку `main`.
* Примерная структура:

  ```
  books_scraper/
  ├── artifacts/
  │   └── books_data.txt
  ├── notebooks/
  │   └── HW_03_python_ds_2025.ipynb
  ├── scraper.py
  ├── README.md
  ├── tests/
  │   └── test_scraper.py
  ├── .gitignore
  └── requirements.txt
  ```